In [5]:
import kagglehub
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torchvision.models import resnet18, ResNet18_Weights
import requests
import json

In [6]:
path = kagglehub.dataset_download("uom190346a/sleep-health-and-lifestyle-dataset")
print("Path to dataset files:", path)
path2 = kagglehub.dataset_download("arsalanjamal002/student-sleep-patterns")
print("Path to dataset files:", path2)

Path to dataset files: C:\Users\destiny\.cache\kagglehub\datasets\uom190346a\sleep-health-and-lifestyle-dataset\versions\2
Path to dataset files: C:\Users\destiny\.cache\kagglehub\datasets\arsalanjamal002\student-sleep-patterns\versions\1


In [7]:
path_image = "D:\Machine_Learning\project\dataset\dataset"
path_text = "D:\Machine_Learning\project\dataset\Sleep_health_and_lifestyle_dataset.csv"

In [8]:
path_new = "D:\Machine_Learning\project\dataset\student_sleep_patterns.csv"
data_text = pd.read_csv(path_text)
data_new = pd.read_csv(path_new)

In [9]:
data_text.columns = [col.strip().replace(" ", "_") for col in data_text.columns]

In [10]:
bp_split = data_text["Blood_Pressure"].str.split("/", expand=True)
data_text["Systolic_BP"] = bp_split[0].astype(float)
data_text["Diastolic_BP"] = bp_split[1].astype(float)
data_text.drop(columns=["Blood_Pressure"], inplace=True)

In [11]:
categorical_cols = ['Gender', 'Occupation', 'BMI_Category', 'Sleep_Disorder']
data_text_encoded = pd.get_dummies(data_text, columns=categorical_cols)

In [12]:
continuous_cols = ['Age', 'Sleep_Duration', 'Quality_of_Sleep',
                   'Physical_Activity_Level', 'Stress_Level',
                   'Heart_Rate', 'Daily_Steps', 'Systolic_BP', 'Diastolic_BP']

scaler = StandardScaler()
data_text_encoded[continuous_cols] = scaler.fit_transform(data_text_encoded[continuous_cols])

In [13]:
x = data_text_encoded.drop(columns=['Person_ID', 'Quality_of_Sleep'])
y = data_text_encoded['Quality_of_Sleep']
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

In [14]:
model_text = RandomForestRegressor(n_estimators=100, random_state=42)
model_text.fit(x_train, y_train)
y_pred = model_text.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse:.2f}")
print(f"R^2: {r2:.2f}")

MSE: 0.02
R^2: 0.98


In [15]:
data_new = pd.get_dummies(data_new, columns=["Gender", "University_Year"], drop_first=False)
data_new = data_new.drop(columns=["Gender_Other"])
feature_new = ["Age", "Sleep_Duration", "Study_Hours", "Screen_Time",
                "Caffeine_Intake", "Physical_Activity",
                "Weekday_Sleep_Start", "Weekend_Sleep_Start",
                "Gender_Male", "Gender_Female",
                "University_Year_2nd Year", "University_Year_4th Year"]
x_new = data_new[feature_new]
y_new = data_new["Sleep_Quality"]

In [16]:
x_train_new, x_test_new, y_train_new, y_test_new = train_test_split(
    x, y, test_size=0.2, random_state=42
)
model_text.fit(x_train_new, y_train_new)
y_pred_new = model_text.predict(x_test_new)

mse_new = mean_squared_error(y_test_new, y_pred_new)
r2_new = r2_score(y_test_new, y_pred_new)

print(f"MSE: {mse_new:.2f}")
print(f"R^2: {r2_new:.2f}")

MSE: 0.02
R^2: 0.98


In [17]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),       
    transforms.ToTensor(),               
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])  
])

In [18]:
class SleepPoseDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.data.columns = [col.strip() for col in self.data.columns] 
        self.image_dir = image_dir
        self.transform = transform
        self.labels_cols = [col for col in self.data.columns if col != 'filename']  

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx]['filename']
        img_path = f"{self.image_dir}/{img_name}"
        image = Image.open(img_path).convert('RGB')
        
        label = int(self.data.iloc[idx][self.labels_cols].values.argmax())
        
        if self.transform:
            image = self.transform(image)
        return image, label


In [19]:
csv_file = "D:\Machine_Learning\project\dataset\dataset\classes.csv"

dataset = SleepPoseDataset(csv_file=csv_file, image_dir=path_image, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
dataframe = pd.read_csv(csv_file)
print(dataframe.columns)

Index(['filename', ' Back', ' Fetal', ' Side', ' Stomach'], dtype='object')


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights = ResNet18_Weights.DEFAULT 
model_image = resnet18(weights=weights)

num_features = model_image.fc.in_features
model_image.fc = nn.Linear(num_features, 4)

model_image = model_image.to(device)

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_image.parameters(), lr=1e-4)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model_image.train()
    running_loss = 0.0
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model_image(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

Epoch 1/10, Loss: 1.1985
Epoch 2/10, Loss: 0.3186
Epoch 3/10, Loss: 0.0952
Epoch 4/10, Loss: 0.0376


In [ ]:
torch.save(model_image.state_dict(), "sleep_pose_model.pth")

In [ ]:
#Zhang, Y., Xiao, A., Zheng, T., Xiao, H., & Huang, R. (2022). The relationship between sleeping position and sleep quality: A flexible sensor-based study. Sensors, 22(16), 6220. https://doi.org/10.3390/s22166220


In [ ]:
pose_weights = {
    "Back": 8,
    "Fetal": 6,
    "Side": 10,
    "Stomach": 5
}

image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

def predict_sleep_score(user_input_1, user_input_2, image_path):
    table_input_1 = pd.DataFrame(user_input_1)
    table_input_1_encoded = pd.get_dummies(table_input_1)
    for col in x_train.columns:
        if col not in table_input_1_encoded.columns:
            table_input_1_encoded[col] = 0
    table_input_1_encoded = table_input_1_encoded[x_train.columns]
    score1 = model_text.predict(table_input_1_encoded)[0]
    score1 = np.clip(score1, 1, 10)

    table_input_2 = pd.DataFrame([user_input_2])
    table_input_2_encoded = pd.get_dummies(table_input_2)
    for col in x_train_new.columns:
        if col not in table_input_2_encoded.columns:
            table_input_2_encoded[col] = 0
    table_input_2_encoded = table_input_2_encoded[x_train_new.columns]
    score2 = model_text.predict(table_input_2_encoded)[0]
    score2 = np.clip(score2, 1, 10)

    image = Image.open(image_path).convert("RGB")
    image_tensor = image_transform(image).unsqueeze(0).to(device)

    model_image.eval()
    with torch.no_grad():
        output = model_image(image_tensor)
        pred_class_idx = torch.argmax(output, dim=1).item()

    classes = ["Back", "Fetal", "Side", "Stomach"]
    pose = classes[pred_class_idx]
    pose_score = pose_weights[pose]

    final_score = 0.4 * score1 + 0.4 * score2 + 0.2 * pose_score
    final_score = np.clip(final_score, 1, 10)

    return final_score, pose, score1, score2, pose_score

user_input = {
    "Gender": ["Male"],
    "Age": [24],
    "Occupation": ["Student"],
    "Sleep_Duration": [8],
    "Physical_Activity_Level": [80],
    "Stress_Level": [2],
    "BMI_Category": ["Normal"],
    "Systolic_BP": [126],
    "Diastolic_BP": [83],
    "Heart_Rate": [77],
    "Daily_Steps": [10000],
    "Sleep_Disorder": ["None"]
}

user_input2 = {
    "Age": 24,
    "Sleep_Duration": 8,
    "Study_Hours": 5,
    "Screen_Time": 5,
    "Caffeine_Intake": 0,
    "Physical_Activity": 80,
    "Weekday_Sleep_Start": 24.0,
    "Weekend_Sleep_Start": 0.5,
    "Gender_Male": 1,    # Male=1, Female=0
    "Gender_Female": 0,
    "University_Year_2nd Year": 1,
    "University_Year_3rd Year": 0,
    "University_Year_4th Year": 0
}

score, pose, score1, score2, pose_score = predict_sleep_score(
    user_input,
    user_input2,
    "D:/Machine_Learning/project/dataset/dataset/5_jpg.rf.53f8c19f946a992507902baabbd8b445.jpg"
)

print(f"final_score: {score:.2f}, pose: {pose}, score1: {score1:.2f}, score2: {score2:.2f} pose_score: {pose_score}")

final_score: 3.06, pose: Side, score1: 1.23, score2: 1.41 pose_score: 10


In [ ]:
url = "http://localhost:11434/v1/chat/completions"
api_key = "bba95d6173f544c381330c270b2111db.xl4EDRCCPqe0l54PJZY2NyWQ"

def clean_user_info(user_input_dict):
    lines = []
    for k, v in user_input_dict.items():
        value = v[0] if isinstance(v, list) else v
        lines.append(f"{k.replace('_',' ')}: {value}")
    return "\n  ".join(lines)

text1 = clean_user_info(user_input)
text2 = clean_user_info(user_input2)

prompt = f"""
System: You are a sleep health expert. Analyze the following user sleep data and provide a detailed evaluation.

User Sleep Analysis:
- Personal Information (Model 1):
  {text1}
- Personal Information (Model 2):
  {text2}
- Table-based predicted score (Model 1): {score1:.2f}
- Table-based predicted score (Model 2): {score2:.2f}
- Image-predicted sleep posture: {pose} (Posture score: {pose_score})
- Final combined sleep score: {score:.2f}

Please provide:
1. Advantages and disadvantages of the user's sleep posture and overall sleep quality
2. Comprehensive recommendations to improve sleep quality
"""

data = {
    "model": "llama3:latest",
    "messages": [{"role": "user", "content": prompt}],
    "temperature": 0.8
}

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

response = requests.post(url, headers=headers, data=json.dumps(data))
result = response.json()

print(result["choices"][0]["message"]["content"])

Based on the provided user sleep data, I'll offer a detailed analysis and suggestions for improvement.

**Advantages and Disadvantages of Sleep Posture:**
The user's sleep posture is recorded as "Side" with a score of 10, indicating a relatively good sleeping position. This posture can be beneficial in several ways:

* Reduces pressure on the spine and joints, which can help alleviate chronic pain and improve overall comfort during sleep.
* Can facilitate better breathing by keeping the airways open.

However, there are some potential drawbacks to consider:

* Sleeping on one's side can lead to uneven support for the body, potentially causing strain on certain muscle groups or joints.
* A side-sleeping position may not be ideal for people who experience acid reflux or snoring issues, as the position can exacerbate these conditions.

**Advantages and Disadvantages of Overall Sleep Quality:**
The user's overall sleep quality is reflected in their final combined sleep score of 3.06. This 